In [ ]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import urllib
from datetime import date

config, context, api_url = postgres_to_girder.config()

girder_connection=postgres_to_girder.connect_to_girder(
    api_url=api_url,
    authentication=(
        config["girder"]["user"],
        config["girder"]["password"]
    )
)
conn = postgres_to_girder.connect_to_postgres(
    config["postgres"]
)

activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="activities",
    girder_connection=girder_connection
)

postgres_tables = {
    table: pd.io.sql.read_sql_query(
        "SELECT * FROM {0};".format(
            table
        ),
        conn
    ) for table in {
        "acts",
        "users",
        "user_acts",
        "organizations",
        "answers"
    }
}

users_emails = postgres_to_girder.postgres_users_to_girder_users(
    postgres_tables["users"],
    girder_connection,
    config["missing_persons"]
)

postgres_tables["acts"] = postgres_to_girder._respondents(
    postgres_tables["acts"]
)

In [ ]:
acts = postgres_tables["acts"]
gc = girder_connection
users = postgres_tables["users"]
activities = postgres_to_girder.postgres_activities_to_girder_activities(
    acts,
    gc,
    api_url,
    users,
    users_emails,
    context
)

In [ ]:
activities